# **Read Images in RGB and convert to LAB**

In [0]:
import numpy as np
import cv2 as cv
import os
from skimage import color
from pathlib import Path
import pandas as pd


def read_rgb(path, list_f):
	everything = np.expand_dims(cv.imread(list_f[0]), axis=0)
	for i in range(1,len(list_f)):
		try:
			img_rgb = cv.imread(list_f[i])
			everything = np.vstack((everything, np.expand_dims(img_rgb, axis=0)))
		except:
			# print(ls[i])
			pass
	# print(everything.shape)
	return everything

def read_lab(path, list_f):
	img_rgb = read_rgb(path, list_f)
	img_lab = color.rgb2lab(img_rgb/255.0)
	return img_lab

def ab_histogram_batch(img_lab):
	img_ab = img_lab[:, :, :, 1:]
	img_ab_vec = np.reshape(img_ab, (np.prod(img_ab.shape[:3]), 2))
	# print(img_ab.shape)
	hist, xedges, yedges = np.histogram2d(img_ab_vec[:, 0], img_ab_vec[:, 1], bins=22, range=[[-110, 110], [-110, 110]])
	return hist, xedges, yedges


def ab_histogram(path):
	c = os.listdir(path)
	ls = []
	for d in c:
		ls += [path + d + "/" + f for f in os.listdir(path + d + "/")]
	# ls = sorted(c)
	i = 0
	hist = np.zeros((22,22))
	while i*1000 < len(ls):
		start = 1000 * i
		end = min(len(ls), 1000 * (i+1))
		list_f = ls[start: end]
		h, xedges, yedges = ab_histogram_batch(read_lab(path, list_f))
		hist += h
		i += 1
		print("#", end = '')

	np.save('./resources/hist.npy', hist)
	np.save('./resources/xedges.npy', xedges)
	np.save('./resources/yedges.npy', yedges)
	pts = []
	probs = []
	color_matrix = np.zeros([22,22, 3]) + 255.0
	for a in range(22):
		for b in range(22):
			lab = np.zeros((1,1,3)) + 50
			lab[:,:,1] = xedges[a]
			lab[:,:,2] = yedges[b]
			rgb = color.lab2rgb(lab)
			if hist[a][b] != 0:
				probs.append(hist[a][b])
				color_matrix[a][b] = rgb * 255.0
				pts.append([xedges[a], xedges[b]])
	pts = np.array(pts).reshape((-1,2))
	total = sum(probs)
	np.save('./resources/pts.npy', pts)
	np.save('./resources/probs.npy', np.array(probs)/total)
	print('done!')

def ab_histogram_single(file, hist_saveto):
    pts = []
    probs = []
    color_matrix = np.zeros([22,22, 3]) + 255.0

    img_rgb = cv.imread(file)
    img_lab = color.rgb2lab(img_rgb/255.0)
    img_ab = img_lab[:, :, 1:]
    img_ab_vec = np.reshape(img_ab, (224 * 224, 2))
    hist, xedges, yedges = np.histogram2d(img_ab_vec[:, 0], img_ab_vec[:, 1], bins=22, range=[[-110, 110], [-110, 110]])
    # print(hist)
    np.save(hist_saveto, hist)
    np.save('./resources/xedges.npy', xedges)
    np.save('./resources/yedges.npy', yedges)
 
    for a in range(22):
        for b in range(22):
            lab = np.zeros((1,1,3)) + 50
            lab[:,:,1] = xedges[a]
            lab[:,:,2] = yedges[b]
            rgb = color.lab2rgb(lab)
            if hist[a][b] != 0:
                probs.append(hist[a][b])
                color_matrix[a][b] = rgb * 255.0
                pts.append([xedges[a], xedges[b]])
    pts = np.array(pts).reshape((-1,2))
    total = sum(probs)
    # np.save('./resources/target_pts.npy', pts)
    # np.save('./resources/target_probs.npy', np.array(probs)/total)
    plt.imshow(color_matrix / 255.0)
    plt.show()
    plt.clf()


# **Convert npy vector to csv file**

In [0]:
def npyVect2csv(npy_file, csv_file):
	data = {'flt': [], 'act_rate': []}
	act_rate = np.load(npy_file)

	for i in range(act_rate.shape[0]):
		data['flt'].append(i)
		data['act_rate'].append(act_rate[i])
	df = pd.DataFrame(data)
	df.to_csv(csv_file, index = False, header=True)
 
def npVect2csv(npV, csv_file):
	data = {'i': [], 'v': []}

	for i in range(npV.shape[0]):
		data['i'].append(i)
		data['v'].append(npV[i])
	df = pd.DataFrame(data)
	df.to_csv(csv_file, index = False, header=True)
 
def top10_to_csv(npy_file, csv_file):
    top10 = np.load('./activation/top10.npy')
    print(top10.shape)
    data = {'x': [], 'y': []}
    for i in range(10):
        data[str(i)] = []
    for x in range(56):
        for y in range(56):
            data['x'].append(x)
            data['y'].append(y)
            for i in range(10):
                data[str(i)].append(top10[i,x,y])
    df = pd.DataFrame(data)
    df.to_csv(csv_file, index = False, header=True)


# **Create Histogram in CSV format from npy array**

In [0]:
def npy2csv_hist(npy_file, csv_file):
    hist = np.load(npy_file)
    xedges = np.load('./resources/xedges.npy')
    yedges = np.load('./resources/yedges.npy')

    data = {'a': [], 'b': [], 'count': [], 'color': []}
    # prob = np.load('./resources/target_probs.npy')

    for a in range(hist.shape[0]):
        for b in range(hist.shape[1]):
            lab = np.zeros((1,1,3)) + 50
            lab[:,:,1] = xedges[a]
            lab[:,:,2] = yedges[b]

            rgb = color.lab2rgb(lab)
            rgb = rgb[0][0] * 255.0
            R,G,B = int(rgb[0]), int(rgb[1]), int(rgb[2])
            hex_col = '#%02x%02x%02x' % (R,G,B)

            # if hist[a][b] != 0:
            data['count'].append(hist[a][b])
            data['color'].append(hex_col)
            data['a'].append(a)
            data['b'].append(b)

    df = pd.DataFrame(data)
    df.to_csv(csv_file, index = False, header=True)


def hist2non_zero_csv(csv_file):
    hist = np.load('./resources/hist.npy')
    xedges = np.load('./resources/xedges.npy')
    yedges = np.load('./resources/yedges.npy')

    data = {'a': [], 'b': [], 'count': [], 'color': []}
    for a in range(hist.shape[0]):
        for b in range(hist.shape[1]):
            lab = np.zeros((1,1,3)) + 50
            lab[:,:,1] = xedges[a]
            lab[:,:,2] = yedges[b]

            rgb = color.lab2rgb(lab)
            rgb = rgb[0][0] * 255.0
            R,G,B = int(rgb[0]), int(rgb[1]), int(rgb[2])
            hex_col = '#%02x%02x%02x' % (R,G,B)

            if hist[a][b] != 0:
                data['a'].append(a)
                data['b'].append(b)
                data['count'].append(int(hist[a][b]))
                data['color'].append(hex_col)
    df = pd.DataFrame(data)
    df.to_csv(csv_file, index = False, header=True)


def hist_from_prob(csv_file):
	hist = np.load('./resources/hist.npy')
	xedges = np.load('./resources/xedges.npy')
	yedges = np.load('./resources/yedges.npy')

	data = {'a': [], 'b': [], 'weight': []}
	prob = np.load('./resources/rebal_probs.npy')

	i = 0
	for a in range(hist.shape[0]):
		for b in range(hist.shape[1]):
			data['a'].append(a)
			data['b'].append(b)

			if hist[a][b] != 0:
				data['weight'].append(prob[i])
				i += 1
			else:
				data['weight'].append(0)
	df = pd.DataFrame(data)
	df.to_csv(csv_file, index = False, header=True)

# **Create Color Map**
color map is a csv file with number of occurence & frequency of each a,b pair

In [0]:
def color_map(csv_file):
	hist = np.load('./resources/hist.npy')
	xedges = np.load('./resources/xedges.npy')
	yedges = np.load('./resources/yedges.npy')

	data = {'a': [], 'b': [], 'count': [], 'color': []}
	for a in range(hist.shape[0]):
		for b in range(hist.shape[1]):
			lab = np.zeros((1,1,3)) + 50
			lab[:,:,1] = xedges[a]
			lab[:,:,2] = yedges[b]

			rgb = color.lab2rgb(lab)
			rgb = rgb[0][0] * 255.0
			R,G,B = int(rgb[0]), int(rgb[1]), int(rgb[2])
			hex_col = '#%02x%02x%02x' % (R,G,B)

			data['a'].append(a)
			data['b'].append(b)
			data['count'].append(int(hist[a][b]))
			data['color'].append(hex_col)
	df = pd.DataFrame(data)
	df.to_csv(csv_file, index = False, header=True)
 

# **Initailize Model Weight with Bias**

In [0]:
def weights_init(m):
	if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
		nn.init.xavier_normal_(m.weight.data)
		nn.init.constant_(m.bias.data, 0.1)

# **Convert Image Batch Shape**

In [0]:
def flatten(pts_nd):
    shape = np.array(pts_nd.shape)
    batch_size = shape[0]
    channels = shape[1]
    img_size = shape[2] * shape[3]

    flat_pts = pts_nd.permute((0, 2, 3, 1))
    flat_pts = flat_pts.contiguous().view(img_size * batch_size, channels)
    return flat_pts


def restore(flat_pts, pts_nd):
    shape = np.array(pts_nd.shape)
    batch_size = shape[0]
    channels = flat_pts.shape[1]
    img_len = shape[2]

    pts_out = flat_pts.reshape([batch_size, img_len, img_len, channels])
    pts_out = pts_out.transpose((0, 3, 1, 2))
    return pts_out